<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
!pip install selenium -q
!pip install webdriver_manager -q

In [ ]:
import pandas as pd
import time
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
Personality_Num = 15 # INTJ
URL=f"https://www.personality-database.com/profile?personality={Personality_Num}"
driver.get(URL)
df = pd.DataFrame(columns=["Character","Contents","Votes","MBTI","Enneagram","category","img_src","Page"])

timeout=60 # element가 화면에 뜰때까지 기다리는 시간
page_cnt=0

while 1:    
    element_present = EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.profile-card')) # 프로필 카드
    cards = WebDriverWait(driver, timeout).until(element_present) # 프로필 카드가 모두 있을 때까지 Wait
    page_cnt+=1 # 첫번째 페이지부터
    next_page = driver.find_element(By.CSS_SELECTOR, "li.rc-pagination-next > button") # 다음 페이지
    print(f"{page_cnt}번째 페이지. {len(cards)}개의 카드정보를 추출합니다.")
    last_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight );")
    before_height = 0
    after_height = last_height/len(cards)
    for card in cards:
        driver.execute_script(f"window.scrollTo({before_height}, {after_height});")
        before_height = after_height
        after_height = after_height + last_height/len(cards)
        MAX_ATTEMPTS = 60
        attempts = 0
        while attempts < MAX_ATTEMPTS:
            try:
                img_element = card.find_element(By.CSS_SELECTOR, ".avatar > img")
                category_element = card.find_element(By.CSS_SELECTOR, ".info-subcategory > img")
                break
            except NoSuchElementException:
                attempts += 1
                time.sleep(1)
        if img_element and category_element:
            img_src = img_element.get_attribute("src")
            category_src = category_element.get_attribute("src")
            df.loc[len(df),:]=card.text.split('\n')+[category_src, img_src, page_cnt]
        else:
            print(f"{card.text[0]} img element was not found after {MAX_ATTEMPTS} attempts")
    if next_page.is_enabled():
        next_page.click()
        time.sleep(10)
    else:
        print("모든 페이지를 크롤링했습니다.")
        break
driver.close()
duplicates = df[df.Character.duplicated(keep=False)].sort_values("Character").reset_index(drop=True)
df.to_csv(f"Personality_Num_{Personality_Num}_{len(df)}_rows.csv",index=False)

In [ ]:
# 중복 Character 확인
duplicates